In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [2]:
import csv
import nltk
import string
import pandas as pd
import numpy as np
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

BUILD THE DATA

In [3]:
import pandas as pd
path = '/Users/ameybasangoudar/Downloads/ArticlesApril2017.csv'
df = pd.read_csv(path)
df.head()

,abstract,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,NaN,58def1347c459f24986d7c80,716,By STEPHEN HILTNER and SUSAN LEHMAN,article,Finding an Expansive View of a Forgotten Peop...,"['Photography', 'New York Times', 'Niger', 'Fe...",3,Insider,2,2017-04-01 00:15:41,Unknown,One of the largest photo displays in Times his...,The New York Times,News,https://www.nytimes.com/2017/03/31/insider/nig...
1,NaN,58def3237c459f24986d7c84,823,By GAIL COLLINS,article,"And Now, the Dreaded Trump Curse","['United States Politics and Government', 'Tru...",3,OpEd,23,2017-04-01 00:23:58,Unknown,Meet the gang from under the bus.,The New York Times,Op-Ed,https://www.nytimes.com/2017/03/31/opinion/and...
2,NaN,58def9f57c459f24986d7c90,575,By THE EDITORIAL BOARD,article,Venezuela’s Descent Into Dictatorship,"['Venezuela', 'Politics and Government', 'Madu...",3,Editorial,22,2017-04-01 00:53:06,Unknown,A court ruling annulling the legislature’s aut...,The New York Times,Editorial,https://www.nytimes.com/2017/03/31/opinion/ven...
3,NaN,58defd317c459f24986d7c95,1374,By MICHAEL POWELL,article,Stain Permeates Basketball Blue Blood,"['Basketball (College)', 'University of North ...",3,Sports,1,2017-04-01 01:06:52,College Basketball,"For two decades, until 2013, North Carolina en...",The New York Times,News,https://www.nytimes.com/2017/03/31/sports/ncaa...
4,NaN,58df09b77c459f24986d7ca7,708,By DEB AMLEN,article,Taking Things for Granted,['Crossword Puzzles'],3,Games,0,2017-04-01 02:00:14,Unknown,In which Howard Barkin and Will Shortz teach u...,The New York Times,News,https://www.nytimes.com/2017/03/31/crosswords/...


In [4]:
corpus = df['snippet'].str.lower().tolist()

# corpus = data.lower().split("\n")
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

total_words = len(tokenizer.word_index) + 1 
word_index = tokenizer.word_index

print(tokenizer.word_index)
print(total_words)

{'the': 1, 'a': 2, 'of': 3, 'and': 4, 'to': 5, 'in': 6, 'for': 7, 'is': 8, 'that': 9, 'with': 10, 'on': 11, 'an': 12, 'be': 13, 'as': 14, 'it': 15, 'are': 16, 'but': 17, 'has': 18, 'new': 19, 'from': 20, 'by': 21, 'at': 22, 'have': 23, 'his': 24, 'about': 25, 'president': 26, 'this': 27, 'what': 28, 'not': 29, 'you': 30, 'who': 31, 'its': 32, 'will': 33, 'they': 34, 'was': 35, 'more': 36, '—': 37, 'one': 38, 'trump': 39, 'how': 40, 'he': 41, 'or': 42, 'her': 43, '”': 44, 'their': 45, 'would': 46, 'some': 47, 'after': 48, 'can': 49, 'people': 50, 'may': 51, 'do': 52, 'york': 53, 'we': 54, 'than': 55, 'us': 56, 'times': 57, 'out': 58, 'like': 59, 'up': 60, 'no': 61, 'could': 62, 'been': 63, 'so': 64, 'when': 65, 'why': 66, 'over': 67, 'if': 68, 'against': 69, 'i': 70, 'many': 71, 'said': 72, 'were': 73, 'into': 74, 'long': 75, 'two': 76, 'years': 77, 'most': 78, 'first': 79, 'which': 80, 'time': 81, 'our': 82, 'way': 83, 'even': 84, 'had': 85, 'year': 86, 'world': 87, 'just': 88, 'say': 

In [5]:
snippets = '\n'.join(df['snippet'])
print(type(snippets))
print(snippets)

<class 'str'>
One of the largest photo displays in Times history, of people who have fled Boko Haram, required an effort just as immense.
Meet the gang from under the bus.
A court ruling annulling the legislature’s authority represents a dangerous turning point for Venezuela.
For two decades, until 2013, North Carolina engaged in academic improprieties that benefited its student-athletes. The repercussions are still being felt.
In which Howard Barkin and Will Shortz teach us a lesson.
In this visually ravishing production, Bobby Cannavale steps into a part that has been waiting for him for decades.
For the author, this gleaming, complex city in British Columbia is “a rolling, improvisatory work in progress.”
About $13 million has been paid out over the years to address complaints from women about Mr. O’Reilly’s behavior. He denies the claims have merit.
Getting rid of mice isn’t easy, but it’s a co-op board’s responsibility if a building becomes infested.
A proposed “border adjustment”

Data Importing and Cleaning

In [6]:
# import re
# snippets = re.sub("&#[0-9]+;", "", snippets)
# snippets = re.sub(",", "", snippets)
# snippets = re.sub("”", "", snippets)
# snippets = re.sub("“", "", snippets)

In [7]:
# alphabets= "([A-Za-z])"
# prefixes = "(Mr|St|Mrs|Ms|Dr|Inc|Ltd|Jr|Sr|Co)[.]"
# suffixes = "(Inc|Ltd|Jr|Sr|Co)"
# starters = "(Mr|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
# acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
# websites = "[.](com|net|org|io|gov|edu|me)"
# digits = "([0-9])"
# def split_into_sentences(text):
#     text = " " + text + "  "
#     text = text.replace("\n"," ")
#     text = re.sub(prefixes,"\\1<prd>",text)
#     text = re.sub(websites,"<prd>\\1",text)
#     text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
#     if "..." in text: text = text.replace("...","<prd><prd><prd>")
#     if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
#     text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
#     text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
#     text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
#     text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
#     text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
#     text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
#     text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
#     if "”" in text: text = text.replace(".”","”.")
#     if "\"" in text: text = text.replace(".\"","\".")
#     if "!" in text: text = text.replace("!\"","\"!")
#     if "?" in text: text = text.replace("?\"","\"?")
#     corpus = text
#     text = text.replace(".",".<stop>")
#     text = text.replace("?","?<stop>")
#     text = text.replace("!","!<stop>")
#     text = text.replace("<prd>",".")
#     sentences = text.split("<stop>")
#     sentences = sentences[:-1]
#     sentences = [s.strip() for s in sentences]
#     return corpus, sentences

In [8]:
# text, sentences = split_into_sentences(snippets)

In [9]:
#Reading Data from text file
# file = open("merged_articles.txt", "r", encoding = "utf8")
# lines = []

# for i in file:
#     lines.append(i)
    
# print("The First Line: ", lines[0])
# print("The Last Line: ", lines[-1])

In [10]:
# data = ""

# for i in lines:
#     data = ' '. join(lines)
    
# data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '')
# data[:500]

In [11]:
# import string
#removing punctuation with space
# translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
# new_data = data.translate(translator)

# new_data[:1000]

In [12]:
# a = []
# # creating a list of unique words by iterating over the data
# for i in data.split():
#     if i not in a:
#         a.append(i)
        
# data = ' '.join(a)
# data[:1000]

Tokenisation

In [18]:
# corpus = text.lower().split('\n')
# print(len(corpus))
# print(type(corpus))
# print(corpus[:2])

In [14]:
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(corpus)
# word_index = tokenizer.word_index
# total_unique_words = len(tokenizer.word_index) + 1 
# print(total_unique_words)
# print(word_index)

In [6]:
for line in corpus:
   seqs = tokenizer.texts_to_sequences([line])[0]
print(seqs)

[1772, 5095, 8, 1, 5096, 5097, 5098, 6, 27, 5099, 340, 546, 11, 1, 5100, 5101, 1195, 582]


Generating N-Grams

In [7]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)): 
      n_gram_seqs = token_list[:i+1]
      input_sequences.append(n_gram_seqs)
print(len(input_sequences))
print(input_sequences[:10])

16230
[[38, 3], [38, 3, 1], [38, 3, 1, 1019], [38, 3, 1, 1019, 696], [38, 3, 1, 1019, 696, 1778], [38, 3, 1, 1019, 696, 1778, 6], [38, 3, 1, 1019, 696, 1778, 6, 57], [38, 3, 1, 1019, 696, 1778, 6, 57, 145], [38, 3, 1, 1019, 696, 1778, 6, 57, 145, 3], [38, 3, 1, 1019, 696, 1778, 6, 57, 145, 3, 50]]


In [8]:
max_seq_length = max([len(x) for x in input_sequences])
input_seqs = np.array(pad_sequences(input_sequences, maxlen=max_seq_length, padding='pre'))
print(max_seq_length)
print(input_seqs[:5])

43
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0   38
     3]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0   38    3
     1]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0   38    3    1
  1019]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0   38    3    1 1019
   696]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0   

In [9]:
x_values, labels = input_seqs[:, :-1], input_seqs[:, -1]
y_values = tf.keras.utils.to_categorical(labels, num_classes=total_words)
print(x_values[:3])
print(labels[:3])

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 38]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 38  3]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 38  3  1]]
[   3    1 1019]


Generate Glove Embeddings

In [10]:
path = '/Users/ameybasangoudar/Downloads/glove.6B.100d.txt'
embeddings_index = {}
with open(path) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.array(values[1:], dtype='float32')
        embeddings_index[word] = coeffs
dict(list(embeddings_index.items())[0:2])

{'the': array([-0.038194, -0.24487 ,  0.72812 , -0.39961 ,  0.083172,  0.043953,
        -0.39141 ,  0.3344  , -0.57545 ,  0.087459,  0.28787 , -0.06731 ,
         0.30906 , -0.26384 , -0.13231 , -0.20757 ,  0.33395 , -0.33848 ,
        -0.31743 , -0.48336 ,  0.1464  , -0.37304 ,  0.34577 ,  0.052041,
         0.44946 , -0.46971 ,  0.02628 , -0.54155 , -0.15518 , -0.14107 ,
        -0.039722,  0.28277 ,  0.14393 ,  0.23464 , -0.31021 ,  0.086173,
         0.20397 ,  0.52624 ,  0.17164 , -0.082378, -0.71787 , -0.41531 ,
         0.20335 , -0.12763 ,  0.41367 ,  0.55187 ,  0.57908 , -0.33477 ,
        -0.36559 , -0.54857 , -0.062892,  0.26584 ,  0.30205 ,  0.99775 ,
        -0.80481 , -3.0243  ,  0.01254 , -0.36942 ,  2.2167  ,  0.72201 ,
        -0.24978 ,  0.92136 ,  0.034514,  0.46745 ,  1.1079  , -0.19358 ,
        -0.074575,  0.23353 , -0.052062, -0.22044 ,  0.057162, -0.15806 ,
        -0.30798 , -0.41625 ,  0.37972 ,  0.15006 , -0.53212 , -0.2055  ,
        -1.2526  ,  0.071624,  

In [11]:
embeddings_matrix = np.zeros((total_words, 100))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector;

Define Validation Metric

In [12]:
import tensorflow as tf
from tensorflow.keras import backend as K
def perplexity(y_true, y_pred):
    cross_entropy = K.categorical_crossentropy(y_true, y_pred)
    return K.exp(K.mean(cross_entropy))

Model Building

In [13]:
from tensorflow.keras import layers
def create_model(use_bidirectional, num_lstm_units, dense_units, dropout_rate, optimizer):
    if not use_bidirectional:
        model = tf.keras.Sequential()

        embed_layer = layers.Embedding(input_dim = total_words, output_dim = 100,
                                   input_length = max_seq_length-1, weights = [embeddings_matrix])
        model.add(embed_layer)

        model.add((layers.LSTM(num_lstm_units, return_sequences=True)))

        model.add(layers.Dropout(dropout_rate))

        model.add((layers.LSTM(num_lstm_units)))

        model.add(tf.keras.layers.Dropout(dropout_rate))

        model.add(tf.keras.layers.Dense(dense_units, activation='relu'))

        model.add(tf.keras.layers.Dense(total_words , activation='softmax'))

        model.compile(optimizer = optimizer, loss = "categorical_crossentropy",  
                  metrics=[perplexity])
    else:
        model = tf.keras.Sequential()

        embed_layer = tf.keras.layers.Embedding(input_dim = total_words, output_dim = 100,
                                                input_length = max_seq_length - 1,
                                                weights = [embeddings_matrix],
                                                trainable = False) # Freeze embedding weights

        model.add(embed_layer)

        model.add(Bidirectional(LSTM(num_lstm_units, return_sequences=True)))

        model.add(Bidirectional(LSTM(num_lstm_units)))

        model.add(Dense(dense_units, activation='relu'))

        model.add(tf.keras.layers.Dropout(dropout_rate))

        # Add dense layer with softmax activation
        model.add(Dense(total_words, activation='softmax'))

        # Compile model
        model.compile(loss='categorical_crossentropy', optimizer = optimizer, metrics=[perplexity])
    return model

LSTM Model

In [14]:
adam = Adam(lr=0.001)
lstm_model = create_model(False, 256, 128, 0.2, adam)
lstm_model.summary()
history = lstm_model.fit(x_values, y_values, epochs=20, validation_split=0.2, verbose=1, batch_size=256)

Metal device set to: Apple M1
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 42, 100)           510200    
                                                                 
 lstm (LSTM)                 (None, 42, 256)           365568    
                                                                 
 dropout (Dropout)           (None, 42, 256)           0         
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dense_1 (Dense)          

Bi-LSTM Model

In [16]:
adam = Adam(lr=0.001)
bilstm_model = create_model(True, 256, 128, 0.2, adam)
bilstm_model.summary()
# Fit model
history = bilstm_model.fit(x_values, y_values, epochs=10, validation_split=0.2, verbose=1)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 42, 100)           510200    
                                                                 
 bidirectional_2 (Bidirectio  (None, 42, 512)          731136    
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 512)              1574912   
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 128)               65664     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 5102)             

Ablation Study 

In [17]:
from sklearn.model_selection import GridSearchCV
# Define the hyperparameters to search over
param_grid = {
    'use_bidirectional':[True, False],
    'num_lstm_units': [32, 64],
    'dense_units': [32, 64],
    'dropout_rate': [0.2, 0.3],
    'optimizer': ['adam', 'rmsprop']
}

In [21]:
# Create the GridSearchCV object
model = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, verbose=1)
grid = GridSearchCV(estimator=model, param_grid=param_grid, verbose = 2, scoring = perplexity)
grid_result = grid.fit(x_values, y_values, epochs = 1)

/var/folders/hw/krvc36_s5f704ddn9wvv0mz80000gn/T/ipykernel_11457/1001719263.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, verbose=1)


Fitting 5 folds for each of 32 candidates, totalling 160 fits
406/406 [==============================] - 40s 84ms/step - loss: 7.5133 - perplexity: 2094.7722
[CV] END dense_units=32, dropout_rate=0.2, num_lstm_units=32, optimizer=adam, use_bidirectional=True; total time=  40.5s
406/406 [==============================] - 40s 85ms/step - loss: 7.5034 - perplexity: 2039.0691
[CV] END dense_units=32, dropout_rate=0.2, num_lstm_units=32, optimizer=adam, use_bidirectional=True; total time=  40.9s
406/406 [==============================] - 40s 85ms/step - loss: 7.5108 - perplexity: 2061.6079
[CV] END dense_units=32, dropout_rate=0.2, num_lstm_units=32, optimizer=adam, use_bidirectional=True; total time=  41.0s
406/406 [==============================] - 41s 87ms/step - loss: 7.5054 - perplexity: 2031.8590
[CV] END dense_units=32, dropout_rate=0.2, num_lstm_units=32, optimizer=adam, use_bidirectional=True; total time=  42.3s
406/406 [==============================] - 41s 87ms/step - loss: 7.503

KeyboardInterrupt: 

In [35]:
# Train the GridSearchCV object
grid_search.fit(x_values, y_values, epochs=50)

# Print the best hyperparameters and accuracy score
print("Best hyperparameters:", grid_search.best_params_)
print("Best accuracy score:", grid_search.best_score_)

TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator <keras.engine.sequential.Sequential object at 0x24f133340> does not.

In [19]:
# #Converting text to sequence after using tokeniser
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts([data])

# # saving the tokenizer for predict function.
# pickle.dump(tokenizer, open('tokenizer1.pkl', 'wb'))

# sequence_data = tokenizer.texts_to_sequences([data])[0]
# sequence_data[:10]

In [20]:
# vocab_size = len(tokenizer.word_index) + 1
# print(vocab_size)

In [21]:
# sequences = [sequence_data[i-1:i+1] for i in range(1, len(sequence_data))]
# print("The length of sequences is: ", len(sequences))
# sequences = np.asarray(sequences)
# print(sequences[:10])